In [735]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F

from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [736]:
df = pd.read_csv("Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [737]:
df = df.drop(["Id"], axis=1)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [738]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [739]:
df['Species'] = df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica':2})
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [740]:
X = df.drop('Species', axis=1).values
y = df['Species'].values

In [742]:
X_train, X_test, y_train, y_test = train_test_split(X y, test_size = 0.2, random_state = 0)

In [743]:
y_train

array([2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 2, 2, 2, 2, 1, 2,
       1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 2, 0, 0, 2, 1, 0, 0,
       1, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2,
       0, 0, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1,
       0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 1, 1, 2, 2, 1, 1, 0, 1, 2, 2, 0, 1,
       1, 1, 1, 0, 0, 0, 2, 1, 2, 0])

In [745]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [746]:
class Model(nn.Module):
    def __init__(self, input_features=4, hidden_layer1=25, hidden_layer2=30, output_features=3):
        super().__init__()
        self.fc1 = nn.Linear(input_features,hidden_layer1)                  
        self.fc2 = nn.Linear(hidden_layer1, hidden_layer2)                  
        self.out = nn.Linear(hidden_layer2, output_features)      
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [747]:
model     = Model(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer.zero_grad()
model

Model(
  (fc1): Linear(in_features=4, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=30, bias=True)
  (out): Linear(in_features=30, out_features=3, bias=True)
)

In [749]:
epochs = 100
losses = []

for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    print(f'epoch: {i:2}  loss: {loss.item():10.8f}')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  0  loss: 1.17416024
epoch:  1  loss: 1.16787493
epoch:  2  loss: 1.15668249
epoch:  3  loss: 1.14224494
epoch:  4  loss: 1.12617254
epoch:  5  loss: 1.10979509
epoch:  6  loss: 1.09410250
epoch:  7  loss: 1.07967746
epoch:  8  loss: 1.06673098
epoch:  9  loss: 1.05511880
epoch: 10  loss: 1.04458034
epoch: 11  loss: 1.03458130
epoch: 12  loss: 1.02447319
epoch: 13  loss: 1.01385963
epoch: 14  loss: 1.00269115
epoch: 15  loss: 0.99108881
epoch: 16  loss: 0.97924072
epoch: 17  loss: 0.96730399
epoch: 18  loss: 0.95529306
epoch: 19  loss: 0.94312727
epoch: 20  loss: 0.93073934
epoch: 21  loss: 0.91811967
epoch: 22  loss: 0.90539539
epoch: 23  loss: 0.89235336
epoch: 24  loss: 0.87885147
epoch: 25  loss: 0.86482656
epoch: 26  loss: 0.85010189
epoch: 27  loss: 0.83489579
epoch: 28  loss: 0.81949341
epoch: 29  loss: 0.80369914
epoch: 30  loss: 0.78766823
epoch: 31  loss: 0.77155870
epoch: 32  loss: 0.75500953
epoch: 33  loss: 0.73804647
epoch: 34  loss: 0.72086883
epoch: 35  loss: 0.7

In [757]:
preds = []
# with torch.no_grad():
for val in X_test:
    y_test_prediction = model.forward(val)
    preds.append(y_test_prediction.argmax().item())

In [751]:
test_df = pd.DataFrame({'Y': y_test, 'Prediction': preds})
test_df['Correct'] = [1 if correct == prediction else 0 for correct, prediction in zip(test_df['Y'], test_df['Prediction'])]

In [752]:
test_df

,Y,Prediction,Correct
0,2,2,1
1,1,1,1
2,0,0,1
3,2,2,1
4,0,0,1
5,2,2,1
6,0,0,1
7,1,1,1
8,1,1,1
9,1,1,1


In [753]:
accuracy = test_df['Correct'].sum() / len(test_df)
print("Accuracy: ", accuracy)


Accuracy:  1.0
